### Customized Email Based on Google Form Response ###

Here, we will use the Google Sheets API to read in responses to a Google Form.  This project needs some setup on the Google Cloud Developer Console as well as some additional Google Cloud libraries for Python.  To get this code running on your own system, here is a helpful resource: https://hackernoon.com/how-to-use-the-google-sheets-api-with-python

pywin32 is a library with many APIs for Microsoft APIS on Windows (I didn't research on MacOS, but I am sure there is a way).  This is used to send an email via Outlook directly frim Python script.

In [ ]:
#!pip install google-cloud
#!pip install google-auth
#!pip install google-auth-oauthlib
#!pip install google-api-python-client

#!pip install pywin32

In [12]:
import pandas as pd
import win32com.client

The block of code below will only run with API credentials.  This connects to the Google Form and reads the data into a pandas dataframe.

In [13]:
from google.oauth2 import service_account
from googleapiclient.discovery import build

# Replace the placeholders with your API credentials
creds = service_account.Credentials.from_service_account_file(
    'gba478-418200-977464f5bc6d.json',
    scopes=['https://www.googleapis.com/auth/spreadsheets.readonly']
)

spreadsheet_id = "12Sui5GzOWmakQmu036pVdWUaf9eqpmU08LRdPq6r8h0"

# Replace the placeholders with your spreadsheet ID and range
service = build('sheets', 'v4', credentials=creds)
sheet = service.spreadsheets()
result = sheet.values().get(spreadsheetId= spreadsheet_id, range='Responses!A1:F1000').execute()

df = pd.DataFrame(result.get('values', []))

row = 1
text = df.iloc[row,1] +" "+ df.iloc[row, 2] +" "+ df.iloc[row,3]
email = df.iloc[row,5]
name = df.iloc[row,4]

In [14]:
text

'New york city is the best. There are shows and people dressed up like my favorite characters. There are even trains underground. San Francisco is a beautiful city. I like to go to the wharf and see the sea lions. Then I climb the hills Florida is a great place to go in winter. They have manatees and alligators. Also oranges.'

In [15]:
email

'Dmohr25@yahoo.com'

In [16]:
import openai
from dotenv import load_dotenv
import os

load_dotenv()
os.environ["OPENAI_API_KEY"] = os.getenv("openai_api_key")

client = openai.OpenAI()

In [17]:
def get_output(prompt, model_input="gpt-4o-mini", temperature_input = 0):
    messages_from_prompt = [{"role": "user", "content": prompt}]
    response = client.chat.completions.create(
        model = model_input,
        messages = messages_from_prompt,
        temperature = temperature_input
    )
    return response.choices[0].message.content

In [18]:
prompt = f"""
You are a friendly travel agent.  Use your client's description of their three favorite trips in the past, summarized in the text {text}, to suggest \
a next destination for them.  Be sure to mention the places they have already visited in your response, so they can see how the recommendation \
is similar to the recommendation you are making.

Craft the response as the body of an email, addressed to the name {name}.  Sign off as "AI TravelBot"
"""
response = get_output(prompt)

In [ ]:
print(response)

THis is the block of code which connects Python to the Outlook application and sends the email.

In [ ]:
ol = win32com.client.Dispatch("outlook.application")
olmailitem = 0x0 #size of the new email
newmail = ol.CreateItem(olmailitem)
newmail.Subject = 'Travel Recommendation from GBA478 AI Chatbot'
newmail.To = email
#newmail.CC='lizamohr1@gmail.com'
newmail.Body = response

# attach='C:\\Users\\admin\\Desktop\\Python\\Sample.xlsx'
# newmail.Attachments.Add(attach)

newmail.Send()